# 💰 Monthly Expense Analysis


## **1. Import Libraries**

In [ ]:
import pandas as pd 


## **2. Load Bank Statement**

In [ ]:
df = pd.read_excel("sample_data/transactions_sample.xlsx")

In [ ]:
df.head()

## **3. Clean Dates & Create Month Column**

In [ ]:
df["Buchungstag"] = pd.to_datetime(df["Buchungstag"], dayfirst=True)


In [ ]:
df["Buchungstag"] = pd.to_datetime(
    df["Buchungstag"],
    format="%d.%m.%y",
    errors="coerce"
)


In [ ]:
df[["Buchungstag", "Month"]].head()


In [ ]:
df.info()


In [ ]:
df["Month"] = df["Buchungstag"].dt.to_period("M")



## **4. Expense Category Rules**

In [ ]:
CATEGORY_RULES = {
    # Grocery
    "EDEKA": "Grocery",
    "REWE": "Grocery",
    "LIDL": "Grocery",
    "PENNY": "Grocery",
    "KAUFLAND": "Grocery",
    "CINAR": "Grocery",
    "MARKAB": "Grocery",
    "ALQUDS": "Grocery",

    # Bakery / Snacks
    "BACKWERK": "Bakery / Snacks",
    "DITSCH": "Bakery / Snacks",
    "BAECKEREI": "Bakery / Snacks",
    "EASTSIDE": "Bakery / Snacks",

    # Restaurants / Cafes
    "BARACHEL": "Restaurants / Cafes",
    "TAZA": "Restaurants / Cafes",
    "DAMASKUS": "Restaurants / Cafes",
    "LATakia": "Restaurants / Cafes",

    # Späti / Kiosk
    "SPATI": "Späti / Kiosk",

    # Clothing
    "NEW YORKER": "Clothing",
    "PRIMARK": "Clothing",
    "UNIQLO": "Clothing",
    "DEICHMANN": "Clothing",
    "WOOLWORTH": "Clothing",

    # Furniture / Home
    "IKEA": "Furniture / Home",
    "POCO": "Furniture / Home",
    "ROLLER": "Furniture / Home",
    "JYSK": "Furniture / Home",

    # Electronics
    "SATURN": "Electronics",

    # DIY
    "TOOM": "DIY / Hardware",

    # Online
    "AMAZON": "Amazon",
    "OTTO": "Online Shopping",

    # Payments
    "PAYPAL": "PayPal / Online Payments",
    "KLARNA": "PayPal / Online Payments",

    # Transport
    "DB VERTRIEB": "Public Transport",

    # Fixed costs
    "VODAFONE": "Internet / Phone",
    "PYUR": "Internet / Phone",
    "FREENET": "Internet / Phone",
    "VATTENFALL": "Electricity / Utilities",

    # Rent
    "WBF": "Rent",

    # Finance
    "BERLINER SPARKASSE": "Bank / Fees",
    "GA NR": "Cash Withdrawal",
    "WESTERN UNION": "Money Transfer",

    # Flowers
    "BLUME2000": "Flowers / Gifts"

    
}


## **5. Income Rules**

In [ ]:
INCOME_RULES = {
    "THE MARCOM ENGINE": "Salary",
    "NEW YORKER": "Salary"
}


## **6. Categorization Logic**

In [ ]:
def categorize_transaction(row):
    text = row["Beguenstigter/Zahlungspflichtiger"]
    amount = row["Betrag"]

    # 1️⃣ Income first
    if amount > 0:
        if pd.notna(text):
            text_upper = text.upper()
            for keyword, category in INCOME_RULES.items():
                if keyword in text_upper:
                    return category
        return "Other Income"

    # 2️⃣ Expenses
    if pd.isna(text):
        return "Bank / Internal"

    text_upper = text.upper()

    for keyword, category in CATEGORY_RULES.items():
        if keyword in text_upper:
            return category

    return "Other Expense"


In [ ]:
df["Category"] = df.apply(categorize_transaction, axis=1)



In [ ]:
df[[
    "Buchungstag",
    "Beguenstigter/Zahlungspflichtiger",
    "Betrag",
    "Category"
]].head(15)


## **7. Monthly Summary**

In [ ]:
monthly_summary = (
    df.groupby(["Month", "Category"])["Betrag"]
      .sum()
      .reset_index()
      .sort_values(["Month", "Betrag"])
)



In [ ]:
display(
    monthly_summary
        .sort_values(["Month", "Betrag"], ascending=[True, True])
)


## **8. Visualization 1: Income vs Expense** 

In [ ]:
df["Month"] = df["Month"].astype(str)


In [ ]:
import os

os.makedirs("exports", exist_ok=True)


In [ ]:
latest_month = monthly_summary["Month"].max()

# Illustrative values (NOT real data)
labels = ["Income", "Expense"]
values = [1, 2]   # just relative placeholders

plt.figure()
plt.bar(labels, values)

plt.title(f"Income vs Expense – {latest_month}")
plt.ylabel("Relative Comparison (masked)")
plt.yticks([])

import os
os.makedirs("exports", exist_ok=True)

plt.savefig(
    f"exports/income_vs_expense_{latest_month}.png",
    dpi=300,
    bbox_inches="tight"
)

plt.show()


## **9. Visualization 2: spending_by_category**

In [ ]:
# Pick one month to visualize
month_to_plot = "2025-11"

plot_df = (
    monthly_summary[monthly_summary["Month"] == month_to_plot]
    .sort_values("Betrag")
)

plt.figure()
plt.barh(plot_df["Category"], plot_df["Betrag"])
plt.title(f"Spending by Category – {month_to_plot}")
plt.xlabel("Amount (€)")
plt.ylabel("Category")

# ✅ CREATE FOLDER FIRST
import os
os.makedirs("exports", exist_ok=True)

# ✅ SAVE IMAGE
plt.savefig(
    f"exports/spending_by_category_{month_to_plot}.png",
    dpi=300,
    bbox_inches="tight"
)
plt.barh(plot_df["Category"], plot_df["Betrag"])
plt.xlabel("Spending")
plt.xticks([])   # removes amounts

plt.show()


## **10. Export to Excel**

In [ ]:
with pd.ExcelWriter("exports/Sparkasse_Monthly_Report.xlsx") as writer:
    
    monthly_summary.to_excel(
        writer,
        sheet_name="Category Summary",
        index=False
    )
    
    monthly_overview.to_excel(
        writer,
        sheet_name="Income_Expense_Net",
        index=False
    )
    
    df.to_excel(
        writer,
        sheet_name="All Transactions",
        index=False
    )


In [ ]:
print("✅ monthly automation finished successfully")
